In [1]:
import time
import pandas as pd
import numpy as np
import math
from math import cos, asin, sqrt
import gc

pd.set_option('display.max_columns', None)

In [2]:
start = time.time()
#########################controlar la excepcion si no esta el fichero creado-----------------------------------------
vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

end = time.time()
print(end - start)

17.0350000858


In [5]:
start = time.time()

if vuelos.empty:
    #hay que tener cuidado de no machacar nada.
    vuelos = pd.read_csv('vuelosDeparted.csv', sep=',',low_memory=False, )
    vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

    vuelos["TMIN_o"]= ''
    vuelos["TMIN_d"]= ''
    vuelos["TMAX_o"]= ''
    vuelos["TMAX_d"]= ''
    vuelos["TAVG_o"]= ''
    vuelos["TAVG_d"]= ''
    vuelos["SNOW_o"]= ''
    vuelos["SNOW_d"]= ''
    vuelos["PRCP_o"]= ''
    vuelos["PRCP_d"]= ''
    vuelos["SNWD_o"]= ''
    vuelos["SNWD_d"]= ''
    vuelos["ACMC_o"]= ''
    vuelos["ACMC_d"]= ''
    vuelos["ACSC_o"]= ''
    vuelos["ACSC_d"]= ''
    vuelos["AWDR_o"]= ''
    vuelos["AWDR_d"]= ''
    vuelos["AWND_o"]= ''
    vuelos["AWND_d"]= ''
    vuelos["EVAP_o"]= ''
    vuelos["EVAP_d"]= ''
    vuelos["FRTH_o"]= ''
    vuelos["FRTH_d"]= ''
    vuelos["TSUN_o"]= ''
    vuelos["TSUN_d"]= ''
    vuelos["WDMV_o"]= ''
    vuelos["WDMV_d"]= ''


    vuelos.to_csv('vuelos.csv', sep=',', index=False)

end = time.time()
print(end - start)

0.000999927520752


In [3]:
start = time.time()
##Leemos el fichero de estaciones de noaa

stationstxt = ""
with open("stations.txt") as input:
    stationstxt = input.read()
    
#Extract the data from file
stations2 = stationstxt.split("\n")
#Remove last line
stations2 = stations2[:-1]
stations2 = map(lambda line: [line[0:11],float(line[13:20]),float(line[22:30]),line[41:71]], stations2)

end = time.time()
print(end - start)

0.404000043869


# Leemos datos del tiempo de noaa

In [30]:
start = time.time()

#weather2012 = pd.read_csv("2012.csv",header=None)
#weather2012.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]

weather2011 = pd.read_csv("2011.csv",header=None)
weather2011.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2014 = pd.read_csv("2014.csv",header=None)
#weather2014.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2015 = pd.read_csv("2015.csv",header=None)
#weather2015.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2017 = pd.read_csv("2017.csv",header=None)
#weather2017.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2016 = pd.read_csv("2016.csv",header=None)
#weather2016.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]


end = time.time()
print(end - start)

89.1619999409


In [5]:
start = time.time()

##nos quedamos con las latitudes de las estaciones.
stationsdf = pd.DataFrame(stations2)
stationsdf.columns = ["id","lat","lng","name"]
lats = stationsdf[['lat','lng']]

#funciones para calcular la distancia entre los aeropuertos y estacion metereologica mas cercana
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v[0],v[1],p[0],p[1]))

#para que la fecha no de problemas al comparar (quizas no este bien del todo)
def tratarFecha(fecha):
    fecha = str(fecha).split(' ')[0]
    fecha = fecha.replace('-','')
    fecha = int(fecha)
    return fecha
end = time.time()
print(end - start)

0.101000070572


In [31]:
start = time.time()
weatherdf = weather2011
del weather2011
del weatherdf['Value3']
del weatherdf['Value4']
del weatherdf['Value5']
del weatherdf['Value2']
weather = weatherdf ##creamos bck
end = time.time()
print(end - start)

8.73900008202


In [7]:
def obtenerEstacion2(row):
    lat = row['board_lat']
    lon = row['board_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lats), coor)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[stationsdf['lat']==EMC[0]]
    aux = stationsdf[stationsdf['lng']==EMC[1]]
    EMC = aux['id']
    s =EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row

def obtenerEstacion3(row):
    lat = row['off_lat']
    lon = row['off_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lats), coor)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[stationsdf['lat']==EMC[0]]
    aux = stationsdf[stationsdf['lng']==EMC[1]]
    EMC = aux['id']
    s =EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row

def aplicarEstacionesOrigen(row):
    row['board_stationid']= coordenadasOrigen[(coordenadasOrigen.board_lat == row['board_lat'])&(coordenadasOrigen.board_lon == row['board_lon'])]['stationid'].values[0]
    return row

def aplicarEstacionesDestino(row):
    row['off_stationid']= coordenadasDestino[(coordenadasDestino.off_lat == row['off_lat'])&(coordenadasDestino.off_lon == row['off_lon'])]['stationid'].values[0]
    return row

In [ ]:
#aplicamos y obtenemos las estaciones (para mejorar el procesado)

########################################################no ejecutar siempre--cntrolar que si existen ya no haga este paso..
coordenadasOrigen = vuelos[['board_lat','board_lon']]
coordenadasDestino = vuelos[['off_lat','off_lon']]
coordenadasOrigen = coordenadasOrigen.drop_duplicates(subset=['board_lat', 'board_lon'])
coordenadasDestino = coordenadasDestino.drop_duplicates(subset=['off_lat', 'off_lon'])
coordenadasOrigen = coordenadasOrigen.apply(lambda x:obtenerEstacion2(x),axis = 1)
coordenadasDestino = coordenadasDestino.apply(lambda x:obtenerEstacion3(x),axis = 1)
vuelos = vuelos.apply(lambda x: aplicarEstacionesOrigen(x),axis = 1)
vuelos = vuelos.apply(lambda x: aplicarEstacionesDestino(x),axis = 1)

In [7]:
def tratar2(row):
    
    board_station = row['board_stationid']
    off_station = row['off_stationid']
    fecha = row['actual_time_of_departure']
    fecha_d = row['actual_time_of_arrival'] 
    ##################################
    #primero tratamos las fechas
    fecha = tratarFecha(fecha)
    fecha_d = tratarFecha(fecha_d)
    #########################################################################################################
    data_board = weatherdf[(weatherdf["id"]==board_station)&(weatherdf['date']==fecha)].sort_values("date")
    data_off   = weatherdf[(weatherdf["id"]==off_station)&(weatherdf['date']==fecha_d)].sort_values("date")
    #####################################################
    data_board["Value1"]=data_board["Value1"]/10
    data_board = data_board.reset_index()
    data_off["Value1"]=data_off["Value1"]/10
    data_off = data_off.reset_index()
        #######################################################
    T1 = data_board[data_board['type']=='TMIN']['Value1']
    T2 = data_off[data_off['type']=='TMIN']['Value1']
    T3 = data_board[data_board['type']=='TMAX']['Value1']
    T4 = data_off[data_off['type']=='TMAX']['Value1']
    T5 = data_board[data_board['type']=='TAVG']['Value1']
    T6 = data_off[data_off['type']=='TAVG']['Value1']
    T7 = data_board[data_board['type']=='SNOW']['Value1']
    T8 = data_off[data_off['type']=='SNOW']['Value1']
    T9 = data_board[data_board['type']=='PRCP']['Value1']
    T10 = data_off[data_off['type']=='PRCP']['Value1']
    T11 = data_board[data_board['type']=='SNWD']['Value1']
    T12 = data_off[data_off['type']=='SNWD']['Value1']
    T13 = data_board[data_board['type']=='ACMC']['Value1']
    T14 = data_off[data_off['type']=='ACMC']['Value1']
    T17 = data_board[data_board['type']=='ACSC']['Value1']
    T18 = data_off[data_off['type']=='ACSC']['Value1']
    T21 = data_board[data_board['type']=='AWDR']['Value1']
    T22 = data_off[data_off['type']=='AWDR']['Value1']
    T23 = data_board[data_board['type']=='AWND']['Value1']
    T24 = data_off[data_off['type']=='AWND']['Value1']
    T39 = data_board[data_board['type']=='EVAP']['Value1']
    T40 = data_off[data_off['type']=='EVAP']['Value1']
    T47 = data_board[data_board['type']=='FRTH']['Value1']
    T48 = data_off[data_off['type']=='FRTH']['Value1']
    T75 = data_board[data_board['type']=='TSUN']['Value1']
    T76 = data_off[data_off['type']=='TSUN']['Value1']
    T89 = data_board[data_board['type']=='WDMV']['Value1']
    T90 = data_off[data_off['type']=='WDMV']['Value1']
    
    
    ##############################
    ##tratamos los valores nulos aplicando un valor nulo
    if T1.empty:
        T1 = 9999
    if T2.empty:
        T2 = 9999
    if T3.empty:
        T3 = 9999
    if T4.empty:
        T4 = 9999
    if T5.empty:
        T5 = 9999
    if T6.empty:
        T6 = 9999
    if T7.empty:
        T7 = 9999
    if T8.empty:
        T8 = 9999
    if T9.empty:
        T9 = 9999
    if T10.empty:
        T10 = 9999
    if T11.empty:
        T11 = 9999
    if T12.empty:
        T12 = 9999
    if T13.empty:
        T13 = 9999
    if T14.empty:
        T14 = 9999
    if T17.empty:
        T17 = 9999
    if T18.empty:
        T18 = 9999
    if T21.empty:
        T21 = 9999
    if T22.empty:
        T22 = 9999
    if T23.empty:
        T23 = 9999
    if T24.empty:
        T24 = 9999
    if T39.empty:
        T39 = 9999
    if T40.empty:
        T40 = 9999
    if T47.empty:
        T47 = 9999
    if T48.empty:
        T48 = 9999
    if T75.empty:
        T75 = 9999
    if T76.empty:
        T76 = 9999
    if T89.empty:
        T89 = 9999
    if T90.empty:
        T90 = 9999
      
        
    #actualizamos la fila
    row['TMIN_o']= float(T1)
    row['TMIN_d']= float(T2)
    row['TMAX_o']= float(T3)
    row['TMAX_d']= float(T4)
    row['TAVG_o']= float(T5)
    row['TAVG_d']= float(T6)
    row['SNOW_o']= float(T7)
    row['SNOW_d']= float(T8)
    row['PRCP_o']= float(T9)
    row['PRCP_d']= float(T10)
    row['SNWD_o']= float(T11)
    row['SNWD_d']= float(T12)
    row['ACMC_o']= float(T13)
    row['ACMC_d']= float(T14)
    row['ACSC_o']= float(T17)
    row['ACSC_d']= float(T18)
    row['AWDR_o']= float(T21)
    row['AWDR_d']= float(T22)
    row['AWND_o']= float(T23)
    row['AWND_d']= float(T24)
    row['EVAP_o']= float(T39)
    row['EVAP_d']= float(T40)
    row['FRTH_o']= float(T47)
    row['FRTH_d']= float(T48)
    row['TSUN_o']= float(T75)
    row['TSUN_d']= float(T76)
    row['WDMV_o']= float(T89)
    row['WDMV_d']= float(T90)


    return row

def functionWeatherDay(year, month, day):
    u = int(month+day)
    d = int(year*10000)
    weatherdf = weather##restauramos el bck
    weatherdf = weatherdf[weatherdf['date']-d==u]
    return weatherdf

def functionPlus(year, month, day):    
    start = time.time()
    vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
     
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]


    x = x.apply(lambda x:tratar2(x),axis = 1)
    #para escribir ya
    #variable auxiliar

    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TMIN_o'] = x.set_index(['index'])['TMIN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_o']).values
    vuelosSinTratar['TMIN_d'] = x.set_index(['index'])['TMIN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_d']).values
    vuelosSinTratar['TMAX_o'] = x.set_index(['index'])['TMAX_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_o']).values
    vuelosSinTratar['TMAX_d'] = x.set_index(['index'])['TMAX_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_d']).values
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    vuelosSinTratar['TAVG_d'] = x.set_index(['index'])['TAVG_d'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_d']).values
    vuelosSinTratar['SNOW_o'] = x.set_index(['index'])['SNOW_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_o']).values
    vuelosSinTratar['SNOW_d'] = x.set_index(['index'])['SNOW_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_d']).values
    vuelosSinTratar['PRCP_o'] = x.set_index(['index'])['PRCP_o'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_o']).values
    vuelosSinTratar['PRCP_d'] = x.set_index(['index'])['PRCP_d'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_d']).values
    vuelosSinTratar['SNWD_o'] = x.set_index(['index'])['SNWD_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_o']).values
    vuelosSinTratar['SNWD_d'] = x.set_index(['index'])['SNWD_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_d']).values
    vuelosSinTratar['ACMC_o'] = x.set_index(['index'])['ACMC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_o']).values
    vuelosSinTratar['ACMC_d'] = x.set_index(['index'])['ACMC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_d']).values
    vuelosSinTratar['ACSC_o'] = x.set_index(['index'])['ACSC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_o']).values
    vuelosSinTratar['ACSC_d'] = x.set_index(['index'])['ACSC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_d']).values
    vuelosSinTratar['AWDR_o'] = x.set_index(['index'])['AWDR_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_o']).values
    vuelosSinTratar['AWDR_d'] = x.set_index(['index'])['AWDR_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_d']).values
    vuelosSinTratar['AWND_o'] = x.set_index(['index'])['AWND_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_o']).values
    vuelosSinTratar['AWND_d'] = x.set_index(['index'])['AWND_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_d']).values
    vuelosSinTratar['EVAP_o'] = x.set_index(['index'])['EVAP_o'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_o']).values
    vuelosSinTratar['EVAP_d'] = x.set_index(['index'])['EVAP_d'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_d']).values
    vuelosSinTratar['FRTH_o'] = x.set_index(['index'])['FRTH_o'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_o']).values
    vuelosSinTratar['FRTH_d'] = x.set_index(['index'])['FRTH_d'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_d']).values
    vuelosSinTratar['TSUN_o'] = x.set_index(['index'])['TSUN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_o']).values
    vuelosSinTratar['TSUN_d'] = x.set_index(['index'])['TSUN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_d']).values
    vuelosSinTratar['WDMV_o'] = x.set_index(['index'])['WDMV_o'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_o']).values
    vuelosSinTratar['WDMV_d'] = x.set_index(['index'])['WDMV_d'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_d']).values


    vuelos['TMIN_o'] = vuelosSinTratar.set_index(['index'])['TMIN_o'].combine_first(vuelos.set_index(['index'])['TMIN_o']).values
    vuelos['TMIN_d'] = vuelosSinTratar.set_index(['index'])['TMIN_d'].combine_first(vuelos.set_index(['index'])['TMIN_d']).values
    vuelos['TMAX_o'] = vuelosSinTratar.set_index(['index'])['TMAX_o'].combine_first(vuelos.set_index(['index'])['TMAX_o']).values
    vuelos['TMAX_d'] = vuelosSinTratar.set_index(['index'])['TMAX_d'].combine_first(vuelos.set_index(['index'])['TMAX_d']).values
    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values
    vuelos['TAVG_d'] = vuelosSinTratar.set_index(['index'])['TAVG_d'].combine_first(vuelos.set_index(['index'])['TAVG_d']).values
    vuelos['SNOW_o'] = vuelosSinTratar.set_index(['index'])['SNOW_o'].combine_first(vuelos.set_index(['index'])['SNOW_o']).values
    vuelos['SNOW_d'] = vuelosSinTratar.set_index(['index'])['SNOW_d'].combine_first(vuelos.set_index(['index'])['SNOW_d']).values
    vuelos['PRCP_o'] = vuelosSinTratar.set_index(['index'])['PRCP_o'].combine_first(vuelos.set_index(['index'])['PRCP_o']).values
    vuelos['PRCP_d'] = vuelosSinTratar.set_index(['index'])['PRCP_d'].combine_first(vuelos.set_index(['index'])['PRCP_d']).values
    vuelos['SNWD_o'] = vuelosSinTratar.set_index(['index'])['SNWD_o'].combine_first(vuelos.set_index(['index'])['SNWD_o']).values
    vuelos['SNWD_d'] = vuelosSinTratar.set_index(['index'])['SNWD_d'].combine_first(vuelos.set_index(['index'])['SNWD_d']).values
    vuelos['ACMC_o'] = vuelosSinTratar.set_index(['index'])['ACMC_o'].combine_first(vuelos.set_index(['index'])['ACMC_o']).values
    vuelos['ACMC_d'] = vuelosSinTratar.set_index(['index'])['ACMC_d'].combine_first(vuelos.set_index(['index'])['ACMC_d']).values
    vuelos['ACSC_o'] = vuelosSinTratar.set_index(['index'])['ACSC_o'].combine_first(vuelos.set_index(['index'])['ACSC_o']).values
    vuelos['ACSC_d'] = vuelosSinTratar.set_index(['index'])['ACSC_d'].combine_first(vuelos.set_index(['index'])['ACSC_d']).values
    vuelos['AWDR_o'] = vuelosSinTratar.set_index(['index'])['AWDR_o'].combine_first(vuelos.set_index(['index'])['AWDR_o']).values
    vuelos['AWDR_d'] = vuelosSinTratar.set_index(['index'])['AWDR_d'].combine_first(vuelos.set_index(['index'])['AWDR_d']).values
    vuelos['AWND_o'] = vuelosSinTratar.set_index(['index'])['AWND_o'].combine_first(vuelos.set_index(['index'])['AWND_o']).values
    vuelos['AWND_d'] = vuelosSinTratar.set_index(['index'])['AWND_d'].combine_first(vuelos.set_index(['index'])['AWND_d']).values
    vuelos['EVAP_o'] = vuelosSinTratar.set_index(['index'])['EVAP_o'].combine_first(vuelos.set_index(['index'])['EVAP_o']).values
    vuelos['EVAP_d'] = vuelosSinTratar.set_index(['index'])['EVAP_d'].combine_first(vuelos.set_index(['index'])['EVAP_d']).values
    vuelos['FRTH_o'] = vuelosSinTratar.set_index(['index'])['FRTH_o'].combine_first(vuelos.set_index(['index'])['FRTH_o']).values
    vuelos['FRTH_d'] = vuelosSinTratar.set_index(['index'])['FRTH_d'].combine_first(vuelos.set_index(['index'])['FRTH_d']).values
    vuelos['TSUN_o'] = vuelosSinTratar.set_index(['index'])['TSUN_o'].combine_first(vuelos.set_index(['index'])['TSUN_o']).values
    vuelos['TSUN_d'] = vuelosSinTratar.set_index(['index'])['TSUN_d'].combine_first(vuelos.set_index(['index'])['TSUN_d']).values
    vuelos['WDMV_o'] = vuelosSinTratar.set_index(['index'])['WDMV_o'].combine_first(vuelos.set_index(['index'])['WDMV_o']).values
    vuelos['WDMV_d'] = vuelosSinTratar.set_index(['index'])['WDMV_d'].combine_first(vuelos.set_index(['index'])['WDMV_d']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [35]:
#variables de inicio
dia = 1
mes = 1
anyo = 2011

In [36]:
for mes in range(1,2):
    print 'empiezo el mes', mes
    mesw = str(mes)
    for dia in range(1,9):
        print 'empiezo el dia - ' ,dia
        diaw = str(dia)
        if dia < 10:
            diaw = '0'+diaw

        weatherdf = functionWeatherDay(anyo,mesw,diaw)
        functionPlus(anyo,mes,dia)

empiezo el mes 1
empiezo el dia -  1
3.76300001144
empiezo el dia -  2
3.62800002098
empiezo el dia -  3
3.58699989319
empiezo el dia -  4
3.64599990845
empiezo el dia -  5
3.62300014496
empiezo el dia -  6
3.56599998474
empiezo el dia -  7
3.53099989891
empiezo el dia -  8
4.32399988174


In [37]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)


In [38]:
#para ver el estado del proceso...borrar luego.
vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
vuelosTratados = vuelos[vuelos['TAVG_o'].notnull()]
print 'sin tratar de todos ', vuelosSinTratar.shape
print 'tratados ', vuelosTratados.shape
x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==2012]
print 'sin tratar totales del año ', x.shape
x = x[x['mesSalida']==mes]
print 'del mes sin tratar ', x.shape
x = x[x['diaSalida']==dia]
print 'vuelos del dia sin tratar ' , x.shape

sin tratar de todos  (0, 86)
tratados  (212698, 86)
sin tratar totales del año  (0, 86)
del mes sin tratar  (0, 86)
vuelos del dia sin tratar  (0, 86)


In [29]:
vuelosSinTratar[vuelosSinTratar['anyoSalida']==2011]

,index,airline_code,flight_number,board_point,board_lat,board_lon,board_country_code,off_point,off_lat,off_lon,off_country_code,distance,scheduled_time_of_departure,actual_time_of_departure,scheduled_time_of_arrival,actual_time_of_arrival,departure_delay,arrival_delay,sched_blocktime,est_blocktime,act_blocktime,aircraft_type,aircraft_registration_number,general_status_code,routing,cabin_1_code,cabin_1_fitted_configuration,cabin_1_saleable,cabin_1_pax_boarded,cabin_1_rpk,cabin_1_ask,cabin_2_code,cabin_2_fitted_configuration,cabin_2_saleable,cabin_2_pax_boarded,cabin_2_rpk,cabin_2_ask,total_rpk,total_ask,load_factor,total_pax,total_no_shows,total_cabin_crew,total_technical_crew,total_baggage_weight,number_of_baggage_pieces,mesSalida,anyoSalida,diaSalida,horaSalida,mesLlegada,anyoLlegada,diaLlegada,horaLlegada,diaSemanaSalida,diaSemanaLlegada,TMIN_o,TMIN_d,TMAX_o,TMAX_d,TAVG_o,TAVG_d,SNOW_o,SNOW_d,PRCP_o,PRCP_d,SNWD_o,SNWD_d,ACMC_o,ACMC_d,ACSC_o,ACSC_d,AWDR_o,AWDR_d,AWND_o,AWND_d,EVAP_o,EVAP_d,FRTH_o,FRTH_d,TSUN_o,TSUN_d,WDMV_o,WDMV_d,board_stationid,off_stationid
212697,222012,OR,776,LEU,42.338611,1.409167,ES,QXB,43.505554,5.367778,FR,347,2011-01-08 12:00:00,2011-01-08 11:56:00,2011-01-08 14:00:00,2011-01-08 13:58:00,-4,-2,120,NaN,122,AT5,MFOAT,Departed,LEU-QXB,Y,48,48,22,7643,16676,NaN,NaN,NaN,NaN,NaN,NaN,7643,16676,0.458333,22,0,1,2,209,19,1,2011,8,11:56:00,1,2011,8,13:58:00,s�bado,s�bado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRM00007627,FR000007650


In [40]:
vuelos.shape

(212698, 86)

In [42]:
vuelos[vuelos.TMIN_d==9999.0].shape

(93590, 86)

In [44]:
vuelos[vuelos.TMIN_o==9999.0].shape

(80587, 86)

In [45]:
vuelos[vuelos.TMAX_d==9999.0].shape

(80129, 86)

In [46]:
vuelos[vuelos.TMAX_o==9999.0].shape

(66248, 86)

In [47]:
vuelos[vuelos.TAVG_d==9999.0].shape

(50849, 86)

In [48]:
vuelos[vuelos.TAVG_o==9999.0].shape

(33124, 86)

In [49]:
vuelos[vuelos.SNOW_o==9999.0].shape

(212687, 86)

In [50]:
vuelos[vuelos.SNOW_d==9999.0].shape

(212691, 86)

In [51]:
vuelos[vuelos.PRCP_o==9999.0].shape

(45673, 86)

In [52]:
vuelos[vuelos.PRCP_d==9999.0].shape

(59212, 86)

In [53]:
vuelos[vuelos.SNWD_o==9999.0].shape

(209910, 86)

In [54]:
vuelos[vuelos.SNWD_d==9999.0].shape

(210240, 86)

In [55]:
vuelos[vuelos.ACMC_o==9999.0].shape

(212698, 86)

In [56]:
vuelos[vuelos.ACMC_d==9999.0].shape

(212698, 86)

In [57]:
vuelos[vuelos.ACSC_o==9999.0].shape

(212698, 86)

In [58]:
vuelos[vuelos.ACSC_d==9999.0].shape

(212698, 86)

In [59]:
vuelos[vuelos.AWDR_o==9999.0].shape

(212698, 86)

In [60]:
vuelos[vuelos.AWDR_d==9999.0].shape

(212698, 86)

In [61]:
vuelos[vuelos.AWND_o==9999.0].shape

(212687, 86)

In [62]:
vuelos[vuelos.AWND_d==9999.0].shape

(212691, 86)

In [63]:
vuelos[vuelos.EVAP_o==9999.0].shape

(212698, 86)

In [64]:
vuelos[vuelos.EVAP_d==9999.0].shape

(212698, 86)

In [65]:
vuelos[vuelos.FRTH_o==9999.0].shape

(212698, 86)

In [66]:
vuelos[vuelos.FRTH_d==9999.0].shape

(212698, 86)

In [67]:
vuelos[vuelos.TSUN_o==9999.0].shape

(212698, 86)

In [68]:
vuelos[vuelos.TSUN_d==9999.0].shape

(212698, 86)

In [69]:
vuelos[vuelos.WDMV_o==9999.0].shape

(212698, 86)

In [70]:
vuelos[vuelos.WDMV_d==9999.0].shape

(212698, 86)

In [72]:
#todas las estaciones
stationsdf.to_csv('stationsAll.csv',sep=',', index=False)

In [77]:
stationstavg = weatherdf[weatherdf["type"]=="TAVG"].groupby("id")["date"].

In [74]:
weatherdf[weatherdf["type"]=="TAVG"]

,id,date,type,Value1
665664,ASN00068151,20110108,TAVG,223
665668,ARM00087582,20110108,TAVG,252
665686,CA002201903,20110108,TAVG,-267
665724,UVM00065518,20110108,TAVG,256
665780,BXM00096315,20110108,TAVG,265
665815,USS0018F01S,20110108,TAVG,-35
665822,USS0019L03S,20110108,TAVG,-45
665829,USS0014G01S,20110108,TAVG,-58
665836,USS0017B04S,20110108,TAVG,-51
665845,USS0014F03S,20110108,TAVG,-71
